In [50]:
import struct

def signed(value: int, width: int):
    """Converts an unsigned value to a signed value."""
    if value >= 2**width:
        raise ValueError(f"Value must be at most {width}-bit{f' ({width//4} bytes)' if width % 4 == 0 else ''}. "
                         f"Your value {value} / {hex(value)} had {len(bin(value))-2} bits.")
    width = width - 1  # example: for four bit number, shift "1" over 4-1=3 to get 1000 (the desired sign bit)
    return -(value & (1 << width)) | (value & ((1 << width) - 1))

hex_value = 'f4f3f2f1'

# Step 1: Convert the hex string to a byte string
byte_data = bytes.fromhex(hex_value)  # b'\xf4\xf3\xf2\xf1'
print(byte_data)

b'\xf4\xf3\xf2\xf1'


In [51]:
# Step 2: Reconstruct the 32-bit word in little-endian order
# Use the unpack function from the struct module to interpret the bytes in little-endian format.
try:
    # '<' = little-endian, 'f4f3f2f1' would first get converted to 'f1f2f3f4' before being converted to int ('>' = big-endian)
    # 'I' = unsigned integer (4-byte, 8 hex chars), 'i' = signed int (4-byte, 8 hex chars)
    # 'H' = unsigned short (2-byte, 4 hex chars), 'h' = signed short (2-byte, 4 hex chars)
    # https://docs.python.org/3/library/struct.html#format-characters
    word_signed: int = struct.unpack('<i', byte_data)[0]  # unpack() returns tuple, here with one element only
    word_unsigned: int = struct.unpack('<I', byte_data)[0]  # unpack() returns tuple, here with one element only
    # word is the signed interpretation of the full 32-bit word, f1f2f3f4 --> -235736076, NOT simply -0xf1f2f3f4
except struct.error:
    print(f"Error: {hex_value} could not be converted to a 32-bit word.")
    raise

print(f"  Signed: {word_signed}, hex: {hex(word_signed)}, binary:    {bin(word_signed)}")
print(f"Unsigned: {word_unsigned}, hex: {hex(word_unsigned)}, binary: {bin(word_unsigned)}")

  Signed: -235736076, hex: -0xe0d0c0c, binary:    -0b1110000011010000110000001100
Unsigned: 4059231220, hex: 0xf1f2f3f4, binary: 0b11110001111100101111001111110100


### NOTE: In the underlying memory, THESE ARE THE SAME  
When you call `bin(x)` on a negative number, it will put a negative sign then just show `bin()` of the positive number  

For example, `bin(3) = 0b11`, `bin(-3) = -0b11` 
 
But in reality in the underlying memory, `3 = 0b0011`, `-3 = 0b1101` (2's complement, -3 is not actually stored as `-0b11`). 

This would almost be the same as `13 = 0b1101`, but Python includes further bits above your bits in your number for sign. For example:

`13 = 0b[0...0]1101`, `-3 = 0b[1...1]1101`, where [0...0] and [1...1] are the infinite extra bits Python adds for sign

so, `-0xE0D0C0C` is the representation of `abs(-235736076)` in hex, with just a negative sign stuck on front  
and, `-0b1110_0000_1101_0000_1100_0000_1100` is the representation of `abs(-235736076)` in binary, with a neg. sign too

Up to 32 bits, `-235736076` = `4059231220`, they are only distinguished starting at the 33rd bit

Equality of the lower 32 bits can be shown: `-235736076 & 0xFFFFFFFF = 4059231220`

the 33rd bit can be shown: `hex(-235736076 & (1 << 32)) = 0x1_0000_0000`, `hex(4059231220 & (1 << 32)) = 0x0_0000_0000`)


In [52]:
print("Lower 32 bits of -235736076 equals 4059231220      :  ", -235736076 & 0xFFFFFFFF, "==", 4059231220)
print("33rd bit for negative number 32-bit number is '1'  :  ", f"{hex(-235736076 & (1 << 32))=}")
print("33rd bit for positive number 32-bit number is '0'  :  ", f"{hex(4059231220 & (1 << 32))=}")

Lower 32 bits of -235736076 equals 4059231220      :   4059231220 == 4059231220
33rd bit for negative number 32-bit number is '1'  :   hex(-235736076 & (1 << 32))='0x100000000'
33rd bit for positive number 32-bit number is '0'  :   hex(4059231220 & (1 << 32))='0x0'


### Because the underlying memory is the same, the following steps will all produce the same results.

In [53]:
# Step 3: Extract the first ADC value (lower 16 bits of `word`)
signed_adc1: int = ((word_signed & 0xFFF0) >> 4)  # Get upper 12 bits, shift right by 4 to be lowest 12 bits
signed_adc1_final = signed(signed_adc1, 12)

unsigned_adc1: int = ((word_unsigned & 0xFFF0) >> 4)  # Get upper 12 bits, shift right by 4 to be lowest 12 bits
unsigned_adc1_final = signed(unsigned_adc1, 12)

print(f"  Signed ADC1: {signed_adc1}, hex: {hex(signed_adc1)}, binary: {bin(signed_adc1)}")
print(f"Unsigned ADC1: {unsigned_adc1}, hex: {hex(unsigned_adc1)}, binary: {bin(unsigned_adc1)}")
print()
print(f"  Signed ADC1 final: {signed_adc1_final}, hex: {hex(signed_adc1_final)}, binary: {bin(signed_adc1_final)}")
print(f"Unsigned ADC1 final: {unsigned_adc1_final}, hex: {hex(unsigned_adc1_final)}, binary: {bin(unsigned_adc1_final)}")

  Signed ADC1: 3903, hex: 0xf3f, binary: 0b111100111111
Unsigned ADC1: 3903, hex: 0xf3f, binary: 0b111100111111

  Signed ADC1 final: -193, hex: -0xc1, binary: -0b11000001
Unsigned ADC1 final: -193, hex: -0xc1, binary: -0b11000001


In [54]:
# Step 4: Extract the second ADC value (upper 16 bits of `word`)
signed_adc2: int = (((word_signed >> 16) & 0xFFF0) >> 4)  # Shift right 16 bits, mask, then shift right 4 again
signed_adc2_final = signed(signed_adc1, 12)

unsigned_adc2: int = (((word_unsigned >> 16) & 0xFFF0) >> 4)  # Shift right 16 bits, mask, then shift right 4 again
unsigned_adc2_final = signed(unsigned_adc1, 12)

print(f"  Signed ADC2: {signed_adc2}, hex: {hex(signed_adc2)}, binary: {bin(signed_adc2)}")
print(f"Unsigned ADC2: {unsigned_adc2}, hex: {hex(unsigned_adc2)}, binary: {bin(unsigned_adc2)}")
print()
print(f"  Signed ADC2 final: {signed_adc2_final}, hex: {hex(signed_adc2_final)}, binary: {bin(signed_adc2_final)}")
print(f"Unsigned ADC2 final: {unsigned_adc2_final}, hex: {hex(unsigned_adc2_final)}, binary: {bin(unsigned_adc2_final)}")

  Signed ADC2: 3871, hex: 0xf1f, binary: 0b111100011111
Unsigned ADC2: 3871, hex: 0xf1f, binary: 0b111100011111

  Signed ADC2 final: -193, hex: -0xc1, binary: -0b11000001
Unsigned ADC2 final: -193, hex: -0xc1, binary: -0b11000001
